In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd "drive/My Drive/Malaria Cell Image Analysis" 
!ls

In [0]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import cv2
import glob
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [0]:
images_1 = glob.glob('Parasitized' + '/*.png')
images_0 = glob.glob('Uninfected' + '/*.png')

In [0]:
print('The image files for Uninfected are: '+str(len(images_0)))
print('The image files for Infected are: '+str(len(images_1)))

In [0]:
# Visualising an example of parasitic image
img = cv2.imread(images_1[445])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.title('infected')
plt.imshow(img)

plt.show()

In [0]:

# Visualising an example of unifected image
img_0 = cv2.imread(images_0[9008])
img_0 = cv2.cvtColor(img_0, cv2.COLOR_BGR2RGB)
plt.title('uninfected')
plt.imshow(img_0)

plt.show()


In [0]:
def load_pic(path):
    """
    image: Argument that represents input image.
    path: Argument that holds the path to the directory and folder where the converted file is to be saved.
    
    Function to convert image into its thresholded version and return the array.
    
    """
    image = cv2.imread(path)
    image = cv2.resize(image, (224,224))
    return image

In [0]:
uninfected_images = []
for path in images_0[:2000]:
    img = load_pic(path)
    uninfected_images.append(img)

In [0]:
infected_images = []
for path in images_1[:2000]:
    img = load_pic(path)
    infected_images.append(img)

In [0]:
images = uninfected_images + infected_images
labels = []
for i in range(len(uninfected_images)):
    labels.append(0)
for j in range(len(infected_images)):
    labels.append(1)

X = np.asarray(images)
y = np.asarray(labels)

idx = np.arange(X.shape[0])
np.random.shuffle(idx)
X = X[idx]
y = y[idx]

#### Train Test Validation Split
Following the 80-20 split convention

In [0]:
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import preprocess_input

In [0]:
# Preprocessing the input data
X = preprocess_input(X)
# Training and Test data split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42) 

#### Conversion to tensor

In [0]:
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Sequential
from keras import backend as K

from keras import optimizers

In [0]:
y_train = np_utils.to_categorical(y_train, num_classes = 2)
y_test = np_utils.to_categorical(y_test, num_classes = 2)

In [0]:
def VGG_Baseline_Model(X_train, X_test):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 
    X_train = base_model.predict(X_train)
    X_test = base_model.predict(X_test)
    print(X_train.shape)
    print(X_test.shape)
    
    return X_train, X_test

In [0]:
# Using the VGG16 model for feature extraction
 train, test = VGG_Baseline_Model(X_train, X_test)

In [0]:
train = train.reshape(train.shape[0],-1)
test = test.reshape(test.shape[0], -1)

Using the features extracted from VGG16 on a simple baseline convolutional n/w we get the final training accuracy as 0.9775

In [0]:
net = Sequential()
net.add(InputLayer((7*7*512,)))    # input layer
net.add(Dense(units=1024, activation='sigmoid')) # hidden layer 1
net.add(Dense(units = 512, activation='sigmoid'))    # hidden layer 2
net.add(Dense(units = 256, activation='sigmoid'))    # hidden layer 3
net.add(Dense(units = 2, activation = 'softmax')) # output layer
net.summary()

In [0]:
net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# Training the model
net.fit(train, y_train, epochs = 50)

In [0]:
net.evaluate(test,y_test)

Yields a minimum accuracy of 0.95 using VGG16 pretrained model for transfer learning.

---


But the following test accuracy is 0.93 with 1000 images from parasitized and uninfected images.

---
Increasing the images to 2000 each- test accuracy becomes 0.9775 with the test accuracy of 0.94.



